In [ ]:
%%time
# colab 기본에서는 못 읽는다. colab pro 정도는 되어야 한다. 고용량 RAM 사용
import pandas as pd
temp = pd.read_csv('/home/q/casey/BigDataSystem/1brc/measurements-004.txt', delimiter=";", names=['location', 'temperature'], header=None)


CPU times: user 2min 56s, sys: 47 s, total: 3min 43s
Wall time: 5min 8s


In [3]:
%%time
temp.groupby(['location']).agg(['min', 'mean', 'max'])

CPU times: user 1min 14s, sys: 14 s, total: 1min 28s
Wall time: 1min 26s


temperature                 
                      min       mean   max
location                                  
Abha                -29.5  18.001155  71.6
Abidjan             -21.9  25.998862  76.3
Abéché              -16.8  29.399406  80.0
Accra               -26.1  26.394080  75.5
Addis Ababa         -34.2  15.992698  68.5
...                   ...        ...   ...
Zagreb              -37.7  10.705807  61.9
Zanzibar City       -20.6  25.997325  78.5
Zürich              -42.5   9.299623  59.5
Ürümqi              -44.2   7.405673  54.6
İzmir               -30.5  17.892632  64.3

[413 rows x 3 columns]

In [4]:
%%time
import duckdb
temp = duckdb.read_csv('/content/drive/MyDrive/BigData/1brc/measurements.txt', delimiter=";", names = ['location', 'temperature'])

CPU times: user 158 ms, sys: 529 ms, total: 687 ms
Wall time: 1.4 s


In [5]:
%%time
duckdb.sql(''' SELECT location, min(temperature), avg(temperature), max(temperature)
               from temp
               group by location''').show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────────────┬──────────────────┬────────────────────┬──────────────────┐
│   location    │ min(temperature) │  avg(temperature)  │ max(temperature) │
│    varchar    │      double      │       double       │      double      │
├───────────────┼──────────────────┼────────────────────┼──────────────────┤
│ Da Nang       │            -19.6 │ 25.794317776838728 │             77.6 │
│ Napier        │            -34.0 │  14.60711256863259 │             68.5 │
│ Lhasa         │            -43.2 │  7.609638145316413 │             56.3 │
│ Kuala Lumpur  │            -21.3 │  27.30678651964085 │             76.0 │
│ Mek'ele       │            -29.2 │  22.69705816072828 │             71.6 │
│ Timbuktu      │            -18.0 │  28.00461024037227 │             76.0 │
│ Garissa       │            -19.6 │ 29.296259483401432 │             79.9 │
│ Cairns        │            -24.5 │ 24.991077445806773 │             73.4 │
│ Winnipeg      │            -46.2 │ 3.0024632034775007 │             50.7 │

In [6]:
%%time
import duckdb
con = duckdb.connect(database=':memory:')

con.execute("""
 COPY (select * FROM read_csv('/content/drive/MyDrive/BigData/1brc/measurements.txt', delim=';', header=False,
 columns = {'location':'VARCHAR','temperature':'float'}))
 TO '/content/drive/MyDrive/BigData/1brc/measurements.parquet'
 (FORMAT PARQUET, CODEC 'SNAPPY');
""")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 8min 20s, sys: 24.3 s, total: 8min 44s
Wall time: 1min 22s


In [2]:
%%time
import duckdb
temp = duckdb.read_parquet('/content/drive/MyDrive/BigData/1brc/measurements.parquet')

CPU times: user 223 ms, sys: 42.7 ms, total: 266 ms
Wall time: 2.58 s


In [3]:
temp

┌──────────────────┬─────────────┐
│     location     │ temperature │
│     varchar      │    float    │
├──────────────────┼─────────────┤
│ Flores,  Petén   │        25.5 │
│ Port Moresby     │        18.5 │
│ Accra            │        24.9 │
│ Abha             │         7.6 │
│ Chicago          │        -0.2 │
│ Kampala          │        24.0 │
│ Seoul            │        18.2 │
│ Reykjavík        │        -3.8 │
│ Lviv             │        17.0 │
│ Hargeisa         │        29.9 │
│    ·             │          ·  │
│    ·             │          ·  │
│    ·             │          ·  │
│ Washington, D.C. │        22.9 │
│ Louisville       │        15.4 │
│ Kunming          │         7.3 │
│ Rostov-on-Don    │        18.9 │
│ Nassau           │        34.0 │
│ Hat Yai          │        19.0 │
│ Yaoundé          │        10.9 │
│ Mexico City      │         6.7 │
│ Napier           │         5.5 │
│ Panama City      │        42.1 │
├──────────────────┴─────────────┤
│ ? rows (>9999 rows

In [4]:
%%time
duckdb.sql(''' SELECT location, min(temperature), avg(temperature), max(temperature)
               from temp
               group by location''').show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────────┬──────────────────┬────────────────────┬──────────────────┐
│  location   │ min(temperature) │  avg(temperature)  │ max(temperature) │
│   varchar   │      float       │       double       │      float       │
├─────────────┼──────────────────┼────────────────────┼──────────────────┤
│ Kandi       │            -21.7 │ 27.701656342694715 │             75.5 │
│ Phoenix     │            -28.1 │ 23.906392906249152 │             73.6 │
│ Tauranga    │            -34.6 │ 14.788784386384213 │             63.6 │
│ Kingston    │            -23.3 │  27.38877133249757 │             77.3 │
│ George Town │            -24.0 │ 27.899197318536302 │             76.0 │
│ Yakutsk     │            -56.2 │  -8.79331310750376 │             42.2 │
│ Makassar    │            -25.2 │  26.70028826944658 │             77.3 │
│ Atlanta     │            -34.7 │ 16.995966168977137 │             66.7 │
│ Nicosia     │            -31.8 │ 19.703818858918424 │             68.8 │
│ Valletta    │          

In [ ]:
%%time
import polars as pl
temp = pl.read_csv('/content/drive/MyDrive/BigData/1brc/measurements.txt',  separator=";",
                    new_columns=["location", "temperature"], has_header=False)

In [ ]:
temp.group_by(["location"]).agg(pl.col("temperature").mean())